In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.metrics import __all__
import re
import time

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.feature_selection import SelectPercentile, chi2

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import  ElasticNet, Lasso, LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import *

from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

import lightgbm as lgbm
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor


import joblib

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("housePrice.csv")


In [ ]:
df.dropna(inplace = True)
df['Area'] = df['Area'].apply(lambda x: re.sub(',', '', x))
df.head()


,Area,Room,Parking,Warehouse,Elevator,Address,Price,Price(USD)
0,63,1,True,True,True,Shahran,1850000000,61666.67
1,60,1,True,True,True,Shahran,1850000000,61666.67
2,79,2,True,True,True,Pardis,550000000,18333.33
3,95,2,True,True,True,Shahrake Qods,902500000,30083.33
4,123,2,True,True,True,Shahrake Gharb,7000000000,233333.33


In [ ]:
# The data matrix
test = df.iloc[:,1:-2]
# The labels
target = (df.iloc[:,-2:-1])

In [ ]:
Columns = test.columns
Columns

Index(['Room', 'Parking', 'Warehouse', 'Elevator', 'Address'], dtype='object')

In [ ]:
target.head()

,Price
0,1850000000
1,1850000000
2,550000000
3,902500000
4,7000000000


In [ ]:
target_column = "Price"
categorical_cols = [c for c in test.columns if test[c].dtype in [object]]
numerical_cols = [c for c in test.columns if test[c].dtype in [float, int] ]
boolean_cols = [c for c in test.columns if test[c].dtype in [bool] ]

print(categorical_cols)
print(numerical_cols)
print(boolean_cols)

['Address']
['Room']
['Parking', 'Warehouse', 'Elevator']


In [ ]:
ohe = OneHotEncoder()


In [ ]:
test[boolean_cols] = test[boolean_cols].astype(int)

In [ ]:
ct = make_column_transformer(
        (ohe, categorical_cols),
        remainder='passthrough')


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    test,
    target,
    test_size=1/3,
    random_state=0)

In [ ]:
lgbm = LGBMRegressor()

In [ ]:
pipe = Pipeline([('ct',ct),('lgbm',LGBMRegressor())])
pipe

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Address'])])),
                ('lgbm', LGBMRegressor())])

In [ ]:
pipe.fit(test,target)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000646 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 100
[LightGBM] [Info] Number of data points in the train set: 3456, number of used features: 48
[LightGBM] [Info] Start training from score 5379956885.240741


Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Address'])])),
                ('lgbm', LGBMRegressor())])

In [ ]:
pipe.predict(X_test)

array([7.91476875e+09, 1.28235580e+09, 2.07027806e+09, ...,
       3.51437584e+09, 9.30501484e+09, 3.74439144e+09])

In [ ]:
print('Training set score: ' + str(pipe.score(X_train,y_train)))
print('Test set score: ' + str(pipe.score(X_test,y_test)))

Training set score: 0.6352833840783312
Test set score: 0.625038944081652


In [ ]:
X_new = [[3,1,1,1,"Pardis"]]
X2 = pd.DataFrame(X_new,columns = Columns )
X2

,Room,Parking,Warehouse,Elevator,Address
0,3,1,1,1,Pardis


In [ ]:
pipe.predict(X2)

array([6.69408679e+09])

In [ ]:
my_model = 'finalized_pipe_model.joblib'
joblib.dump(pipe, my_model)

['finalized_pipe_model.joblib']

In [ ]:
model = joblib.load('finalized_pipe_model.joblib')
# Address,Area,Parking,Elevator,Warehouse
Columns = [ "Area", "Room", "Parking", "Warehouse", 'Elevator',"Address"]
Address = "Darakeh"
Area = 100000
Room = 3
Parking = "Yes"
Elevator = "No"
Warehouse = "Yes"

Elevator = int(Elevator == "Yes")
Parking = int(Parking == "Yes")
Warehouse = int(Warehouse == "Yes")

final_vec = [Area, Room, Parking, Warehouse, Elevator, Address]
#row = np.array(final_vec)
X = pd.DataFrame([final_vec], columns = Columns)
prediction = model.predict(X)
print(prediction)


[7.67434112e+09]
